In [1]:
import numpy as np
from collections import defaultdict
import nltk 
import pandas as pd
import os 
import json
import io
import gzip

The code shown below is from DSC3 Datathon

In [2]:
import random
import ast

References:
- random sampler fast: http://metadatascience.com/2014/02/27/random-sampling-from-very-large-files/
- decode the dict: https://stackoverflow.com/questions/49184578/how-to-convert-bytes-type-to-dictionary
- stops: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [3]:
#have a random sample so it's more representative of the population
random.seed(42)
def random_sampler(filename, k):
    sample = []
    with open(filename, 'rb') as f:
        f.seek(0, 2)
        filesize = f.tell()
        random_set = sorted(random.sample(range(filesize), k))

        for i in range(k):
            f.seek(random_set[i])
            # Skip current line (because we might be in the middle of a line) 
            f.readline()
            # Append the next line to the sample set 
            val = ast.literal_eval(f.readline().rstrip().decode('utf-8'))
            sample.append(val)

    return sample

In [4]:
places2 = random_sampler("places.clean.json", 2000)


In [ ]:
%%time
#parsing in the data, takes time
places = random_sampler("places.clean.json", 200000)
users = random_sampler("users.clean.json", 200000)
reviews = random_sampler("reviews.clean.json", 200000)

In [ ]:
#making the dict into dataframes
placesdf = pd.DataFrame(places)
usersdf = pd.DataFrame(users)
reviewsdf = pd.DataFrame(reviews)

In [ ]:
full = placesdf.merge(reviewsdf,how = "outer")

In [ ]:
len(full)

In [ ]:
#get rid of rows without address
#df = full[full["address"].notna()]
df = full[full["rating"].notna()]
df = df[df["reviewText"].notna()]
#df2 = df[df["price"].notna()]#

In [ ]:
len(df)

Now we have a df with the rating and the values, we're going to translate into english, but there's no need to translate the already english ones, so I'm removing the reviews with english stop words and only translating the ones without english stop words.

In [ ]:
import nltk
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [ ]:
#scoring english based on how many english stop words there are
df["english score"] = df["reviewText"].apply(lambda x: sum([int(term in stops) for term in x.split()]))

In [ ]:
from langdetect import detect

In [ ]:
from collections.abc import Iterable

In [ ]:
isinstance(np.nan, Iterable)
str(np.nan)

In [ ]:
def remove_from_list(x):
    """function to remove lists from the dataframe, making the address into
    a string instead"""
    if isinstance(x, Iterable):
        return " ".join(x)
    else:
        return str(x)

In [ ]:
def remove_from_list_geo(x):
    """a modified remove_from_list to modify geo data 
    to get long and lat into a tuple"""
    if isinstance(x, Iterable):
        return tuple(x)
    else:
        return str(x)

In [ ]:
"".join(["hi this is a list"])

In [ ]:
#df["address"] = 
df["address"] = df["address"].apply(remove_from_list)

In [ ]:
df["gps"] = df["gps"].apply(remove_from_list_geo)

In [ ]:
#df["categories"] = 
df["categories"]= df["categories"].apply(remove_from_list)

In [ ]:
clean = df.drop(["hours"], axis =1)

In [ ]:
new_df= clean.drop_duplicates()

In [ ]:
len(new)

In [ ]:
detect("half of a glass")

In [211]:
def english(text):
    """The langdetect doesn't work with empty strings 
    and numbers, use a if else to try and avoid error"""
    try:
        return detect(text)
    except:
        return "unknown"
        

In [ ]:
#new["language"] = new["reviewText"].apply(english)

In [109]:
df["language"] = df["reviewText"].apply(lambda x: detect(str(x)))

LangDetectException: No features in text.

In [105]:
df[df["english score"] < 2]

,address,closed,gPlusPlaceId,gps,hours,name,phone,price,categories,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,english score
40,"[San Martín 3400, 3400 Corrientes, Argentina]",False,118235232758603031011,"[-27.471143, -58.82315]",None,General Belgrano Bridge,03783 42-7200,None,[Bridge],105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09,1
42,"[San Martín 3400, 3400 Corrientes, Argentina]",False,118235232758603031011,"[-27.471143, -58.82315]",None,General Belgrano Bridge,03783 42-7200,None,[Bridge],105611792260212147994,2.0,La dirección proporcionada por Google Maps no ...,"Dec 16, 2013",julio taborda,1.387228e+09,1
123,"[Mueang Khon Kaen District, Khon Kaen, Thailand]",False,101466126288249455760,"[16.413289, 102.838298]",None,บึงแก่นนคร,None,None,[Tourist Attraction],102595007700026792843,3.0,สามดาวพอนะ เพราะบางที เเม่งขี้เดา พากู หลงเส้น...,"Oct 3, 2013",Nitchkarn Seebunruang,1.380786e+09,0
126,"[Đường Không Tên, Giang Điền, Trảng Bom, Đồng ...",False,101927452218854067749,"[10.912831, 106.986051]",None,Khu Du lịch Sinh thái thác Giang Điền,061 3923 930,None,[Du Lịch - Du Lịch Sinh Thái],110208324157728215085,5.0,Tot qua,"Jul 30, 2013",Tram Phan,1.375188e+09,0
173,"[W Tropicana Ave, Las Vegas, NV 89147]",False,101762032948767959677,"[36.100717, -115.302022]","[[Monday, [['10:30 am--1:00 am']], 1], [Tuesda...",In-N-Out Burger,(800) 786-1000,$$$,"[Hamburger Restaurant, Fast Food Restaurant, B...",114543247800188081386,5.0,Awesome stuff!,"Jul 27, 2013",Samantha Walker,1.374959e+09,0
307,"[Rua Camboriú, 647, SC, 88301-450, Brazil]",False,112310896318623723556,"[-26.913699, -48.655571]",None,Beth Bistro e Restaurante,(47) 3444-5516,None,None,114907615512867408757,5.0,"Ambiente muito agradável, ótimo atendimento e ...",None,André A.,NaN,0
324,"[H.L.M 4, Dakar, Senegal]",False,111510704075044006045,"[14.706486, -17.442727]",None,Ecole HLM4 C/D,None,None,[School],112735649947939669150,5.0,J adore,"Sep 3, 2013",Ndeye Gnima Mandiang,1.378247e+09,0
397,"[Via Lavoratori Autobianchi, 1, 20033 Desio MB...",False,111532397928835458938,"[45.62842, 9.211563]","[[Monday, [['12:00--3:00 pm'], ['7:00 pm--1:00...",Eurotaverna,0362 300046,None,[Restaurant],115741219409509291006,5.0,È molto bello io ci vado sempre,"Apr 14, 2013",Camilla Casorati,1.365963e+09,0
560,"[Al Rawdah, Al Faisaliyah, Jeddah 23442, Saudi...",False,116378342621164987320,"[21.567553, 39.180532]",None,شيزان,02 663 5180,None,[Indian Restaurant],106494967655449429160,5.0,فيه شباب ولا بس عوائل,"Sep 21, 2013",alwaleed bawghash,1.379770e+09,0
608,"[Japan, 〒163-0248 Tokyo, Shinjuku, Nishishinju...",False,107255972992272619664,"[35.691296, 139.692623]","[[Monday, [['9:30 am--5:30 pm']]], [Tuesday, [...",平和祈念展示資料館,03-5323-8709,None,None,102881488089763968454,4.0,戦渦の、そして戦争直後に、日本人が辿ってきた過酷な日々と筆舌しがたい労苦を今に語り継ぐ資料館...,None,Hiroaki Kaneko,NaN,0


In [104]:
df["english score"]

9           4
23         13
40          1
42          1
89         76
95         49
115         2
118        38
123         0
126         0
173         0
211        23
218        13
258        23
273       120
307         0
324         0
368         9
369         8
371         3
372        30
397         0
486        24
518        18
524        12
560         0
578        27
579        32
580        32
581        52
         ... 
389167     14
389168      4
389169      0
389171      1
389172     39
389173      0
389174      0
389175     38
389177      3
389178     21
389179      1
389180      1
389182      1
389183     47
389184      0
389185     48
389186      0
389187     31
389188      0
389189     24
389190     90
389191      4
389192      0
389193     28
389194     17
389195      5
389196     33
389197     12
389198     62
389199    101
Name: english score, Length: 149350, dtype: int64

Using Google's API to translate text.

In [9]:
import google.cloud.translate 

https://stackoverflow.com/questions/45501082/set-google-application-credentials-in-python-project-to-use-google-api

In [20]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Datathon-ad4a6dac1782.json"

In [23]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

# The text to translate
text = u'很多料加在同一碗粥裡，賣的很貴，但味道根本不搭。'
# The target language
target = 'en'

# Translates some text into Russian
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))

Text: 很多料加在同一碗粥裡，賣的很貴，但味道根本不搭。
Translation: A lot of materials are added to the same bowl of porridge, which is very expensive to sell, but the taste is not at all.
